In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from google.colab import drive

In [5]:



fn = pd.read_csv("food_nutrient.csv", usecols=['fdc_id', 'nutrient_id', 'amount'])

fn.head().tail()

,fdc_id,nutrient_id,amount
0,1105904,1003,0.00
1,1105904,1004,93.33
2,1105904,1005,0.00
3,1105904,1110,0.00
4,1105904,1079,0.00


In [7]:
f = pd.read_csv("food.csv")

f.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13
1,1105905,branded_food,SWANSON BROTH BEEF,NaN,2020-11-13
2,1105906,branded_food,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,NaN,2020-11-13
3,1105907,branded_food,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,NaN,2020-11-13
4,1105898,experimental_food,Discrepancy between the Atwater factor predict...,NaN,2020-10-30


In [8]:
n = pd.read_csv("nutrient_incoming_name.csv",usecols=['nutrient_id', 'name'])


In [9]:
n.head()

,name,nutrient_id
0,NITROGEN-DUMAS METHO,1002
1,Nitrogen,1002
2,NITROGEN-DUMAS METHOD,1002
3,Nitrogen - Kjeldahl,1002
4,Protein,1003


# merging food_nutrient and nutrient_incoming_name on 'nutrient_id'

In [10]:
merged_df = pd.merge(fn, n, on='nutrient_id')

In [11]:
merged_df.head()

,fdc_id,nutrient_id,amount,name
0,1105904,1003,0.0,Protein
1,1105904,1003,0.0,Proteins
2,1105904,1003,0.0,PROTEIN (N X 5.70) D
3,1105904,1003,0.0,PROTEIN (N X 6.38) D
4,1105904,1003,0.0,PROTEIN(N X 6.25)DUM


In [12]:
merged_df.tail()

,fdc_id,nutrient_id,amount,name
132847642,2352625,1032,0.0,CITRIC ACID
132847643,2352650,1032,0.0,CITRIC ACID
132847644,2352675,1032,0.0,CITRIC ACID
132847645,2352701,1032,0.0,CITRIC ACID
132847646,2352734,1032,0.0,CITRIC ACID


# merging merged_df and food on 'fdc_id'

In [13]:
merged_df2 = pd.merge(f, merged_df, on='fdc_id')
merged_df2.head()

,fdc_id,data_type,description,food_category_id,publication_date,nutrient_id,amount,name
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1003,0.0,Protein
1,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1003,0.0,Proteins
2,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1003,0.0,PROTEIN (N X 5.70) D
3,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1003,0.0,PROTEIN (N X 6.38) D
4,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1003,0.0,PROTEIN(N X 6.25)DUM


In [14]:
merged_df2.tail()

,fdc_id,data_type,description,food_category_id,publication_date,nutrient_id,amount,name
132847642,2353553,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,1.58,"Fiber, Dietary, Total"
132847643,2353553,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,1.58,Dietary Fiber
132847644,2353553,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,1.58,Total Dietary Fiber (TDF)
132847645,2353553,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,1.58,TDF
132847646,2353553,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,1.58,Fiber


# filtering data on nutrients name.

In [15]:
filtered_result = merged_df2[merged_df2['name'].isin(['Protein','FAT','Carbohydrate','Fat, saturated','Fiber','CALORIES','TOTAL TRANS FATTY ACID','Starch','TOTAL SUGAR',
                                                     'Lactose','Caffeine','Vitamin A','Vitamin D','Vitamin E','Vitamin C','Vitamin B12','Caffeine','Iron'])]
filtered_result.head()

,fdc_id,data_type,description,food_category_id,publication_date,nutrient_id,amount,name
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1003,0.00,Protein
21,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1004,93.33,FAT
44,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1005,0.00,Carbohydrate
47,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1110,0.00,Vitamin D
56,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13,1079,0.00,Fiber


# converting nutrient name to columns on the filtered data

In [16]:
df_pivot = filtered_result.pivot(index=['fdc_id','data_type','description','food_category_id','publication_date','nutrient_id'], columns='name', values='amount').reset_index()
df_pivot.head()

name,fdc_id,data_type,description,food_category_id,publication_date,nutrient_id,CALORIES,Caffeine,Carbohydrate,FAT,...,Iron,Lactose,Protein,Starch,TOTAL TRANS FATTY ACID,Vitamin A,Vitamin B12,Vitamin C,Vitamin D,Vitamin E
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,5.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1004,NaN,NaN,NaN,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1005,NaN,NaN,41.18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1008,307.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
row_count = len(df_pivot)
row_count

13799490

In [18]:
new_column_names = {'name': 'id', 'fdc_id': 'food_id', 'data_type': 'food_data_type', 'description':'food_description'}
df = df_pivot.rename(columns=new_column_names)

In [19]:
df.head()

name,food_id,food_data_type,food_description,food_category_id,publication_date,nutrient_id,CALORIES,Caffeine,Carbohydrate,FAT,...,Iron,Lactose,Protein,Starch,TOTAL TRANS FATTY ACID,Vitamin A,Vitamin B12,Vitamin C,Vitamin D,Vitamin E
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,5.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1004,NaN,NaN,NaN,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1005,NaN,NaN,41.18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1008,307.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
row_count = len(df)
df

name,food_id,food_data_type,food_description,food_category_id,publication_date,nutrient_id,CALORIES,Caffeine,Carbohydrate,FAT,...,Iron,Lactose,Protein,Starch,TOTAL TRANS FATTY ACID,Vitamin A,Vitamin B12,Vitamin C,Vitamin D,Vitamin E
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,5.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1004,NaN,NaN,NaN,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1005,NaN,NaN,41.18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1008,307.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13799485,2353549,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13799486,2353550,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13799487,2353551,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13799488,2353552,sub_sample_food,"MUSHROOMS, WHITE BUTTON",NaN,2022-10-28,1079,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
grouped_df = df.groupby('food_id')


In [22]:
sum_df = grouped_df.sum()
sum_df.head()

/var/folders/s8/l77vf42d6qsg12wpvgk1p38h0000gr/T/ipykernel_26188/1245968499.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_df = grouped_df.sum()


name,food_category_id,nutrient_id,CALORIES,Caffeine,Carbohydrate,FAT,"Fat, saturated",Fiber,Iron,Lactose,Protein,Starch,TOTAL TRANS FATTY ACID,Vitamin A,Vitamin B12,Vitamin C,Vitamin D,Vitamin E
food_id,,,,,,,,,,,,,,,,,,
167512,0.0,8703,307.0,0.0,41.18,13.24,2.941,1.2,2.12,0.00,5.88,0.00,4.412,0.0,0.00,0.0,0.0,0.0
167513,0.0,10969,330.0,0.0,53.42,11.27,3.250,1.4,1.93,0.00,4.34,0.00,4.290,1.0,0.00,0.1,0.0,0.0
167514,0.0,4020,377.0,0.0,79.80,3.70,0.000,0.0,0.00,0.00,6.10,0.00,0.000,0.0,0.00,0.0,0.0,0.0
167515,0.0,8728,232.0,0.0,46.00,1.80,0.308,0.0,1.40,0.00,8.00,0.00,0.082,0.0,0.00,0.0,0.0,0.0
167516,0.0,13969,273.0,0.0,41.05,9.22,1.898,2.2,6.04,1.35,6.58,35.51,0.000,1340.0,2.86,0.0,0.0,0.0


In [23]:
unique_rows = df.drop_duplicates(subset=['food_id','food_data_type','food_description'])
unique_rows.head()

name,food_id,food_data_type,food_description,food_category_id,publication_date,nutrient_id,CALORIES,Caffeine,Carbohydrate,FAT,...,Iron,Lactose,Protein,Starch,TOTAL TRANS FATTY ACID,Vitamin A,Vitamin B12,Vitamin C,Vitamin D,Vitamin E
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,5.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,4.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,6.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,8.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",NaN,2019-04-01,1003,NaN,NaN,NaN,NaN,...,NaN,NaN,6.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
unique_rows = unique_rows.drop(columns=['food_category_id','publication_date','nutrient_id','Carbohydrate','FAT','Fat, saturated','Fiber','Protein'])
unique_rows.head()

name,food_id,food_data_type,food_description,CALORIES,Caffeine,Iron,Lactose,Starch,TOTAL TRANS FATTY ACID,Vitamin A,Vitamin B12,Vitamin C,Vitamin D,Vitamin E
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
final = pd.merge(unique_rows, sum_df, on='food_id')
final.head()

name,food_id,food_data_type,food_description,CALORIES_x,Caffeine_x,Iron_x,Lactose_x,Starch_x,TOTAL TRANS FATTY ACID_x,Vitamin A_x,...,Iron_y,Lactose_y,Protein,Starch_y,TOTAL TRANS FATTY ACID_y,Vitamin A_y,Vitamin B12_y,Vitamin C_y,Vitamin D_y,Vitamin E_y
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.12,0.00,5.88,0.00,4.412,0.0,0.00,0.0,0.0,0.0
1,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.93,0.00,4.34,0.00,4.290,1.0,0.00,0.1,0.0,0.0
2,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,6.10,0.00,0.000,0.0,0.00,0.0,0.0,0.0
3,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.40,0.00,8.00,0.00,0.082,0.0,0.00,0.0,0.0,0.0
4,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.04,1.35,6.58,35.51,0.000,1340.0,2.86,0.0,0.0,0.0


In [26]:
final = final.drop('food_category_id',axis=1)
final.head()

name,food_id,food_data_type,food_description,CALORIES_x,Caffeine_x,Iron_x,Lactose_x,Starch_x,TOTAL TRANS FATTY ACID_x,Vitamin A_x,...,Iron_y,Lactose_y,Protein,Starch_y,TOTAL TRANS FATTY ACID_y,Vitamin A_y,Vitamin B12_y,Vitamin C_y,Vitamin D_y,Vitamin E_y
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.12,0.00,5.88,0.00,4.412,0.0,0.00,0.0,0.0,0.0
1,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.93,0.00,4.34,0.00,4.290,1.0,0.00,0.1,0.0,0.0
2,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,6.10,0.00,0.000,0.0,0.00,0.0,0.0,0.0
3,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.40,0.00,8.00,0.00,0.082,0.0,0.00,0.0,0.0,0.0
4,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.04,1.35,6.58,35.51,0.000,1340.0,2.86,0.0,0.0,0.0


In [27]:
row_count = len(final)
row_count

1613224